### Retrieval-Augmented Generation (RAG) with Large Language Models(LLMs)

RAG models combine the power of language models with external knowledge sources to provide more informed and accurate responses.

To implement the RAG Systems with LLMs the following steps were applied:

1. Data preparation:
- Study the data source and prepare the text for work. Divide the text into semantic parts (chunks) so that each chunk contains a complete thought or information block. The purpose of this step is to facilitate the process of text vectorization and improve the quality of information extraction.

2. Choosing an LLM model and a vector database:
- Determine which model from the Large Language Models family you will use to generate and vectorize text. Justify your choice.
- Select a suitable vector database for storing and retrieving embeddings. Explain why it is suitable for your task.

3. Implementation of the embedding extraction mechanism:
- Develop a process for vectorizing text chunks using the selected LLM model.
- Save the received embeddings in a vector database.
- Implement a mechanism for searching and extracting information from a database based on vector search. The system must accept the request, convert it into a vector, find the most relevant chunks and generate a response based on them.

4. System integration and testing:
- Integrate system components into a single architecture inside Jupyter Notebook.
- Test the system with examples of queries related to the content of the source document. Evaluate the quality of the responses you receive and make the necessary adjustments to the system.

### Notebook imports

In [95]:
import os
import fitz
import requests
from tqdm import tqdm
import random
import pandas as pd

### CONSTANTS

In [83]:
FILE_PATH = 'TECHNOLOGICAL AND NUCLEAR SUPERVISION.pdf'

### 1. Data Preparation

#### 1.1 import PDF Document

In [84]:
# Downloading the document
if not os.path.exists(FILE_PATH):
    print(f'[INFO] File not found. Downloading...')
    
    # PDF url
    url = "https://github.com/MamvotaTake/RAG-with-LLMs/blob/master/System/TECHNOLOGICAL AND NUCLEAR SUPERVISION.pdf"
    
    response = requests.get(url)
    
    filename = FILE_PATH
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f'[INFO] The file has been downloaded and saved as {filename}')
    else:
        print(f'[ERROR] The file could not be downloaded. Status code: {response.status_code}')
    
else:
    print(f'[INFO] The file {FILE_PATH} already exists')

[INFO] The file TECHNOLOGICAL AND NUCLEAR SUPERVISION.pdf already exists


### Reading the PDF

In [126]:
def text_formatter(text: str) -> str:
    """Text formatting"""
    cleaned_text = text.replace("\n", " ").strip()
    cleaned_text = cleaned_text.replace("--------------------", " ").strip()

    
    return cleaned_text


In [127]:
def open_and_read_pdf(file_path: str) -> list[dict]:
    document = fitz.open(file_path)
    
    
    
    pages_and_text = []
    
    for page_number, page in tqdm(enumerate(document)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_text.append({
            'page_number': page_number + 1, 
            'page_char_count': len(text), 
            'page_word_count':len(text.split(" ")),
            'page_sentence_count_row': len(text.split(". ")),
            'page_token_count': len(text) / 4,
            "text": text})
    return pages_and_text

In [128]:
pages_and_texts = open_and_read_pdf(file_path=FILE_PATH)
pages_and_texts[:2]

12it [00:00, 89.44it/s]


[{'page_number': 1,
  'page_char_count': 1362,
  'page_word_count': 223,
  'page_sentence_count_row': 16,
  'page_token_count': 340.5,
  'text': 'Зарегистрировано в Минюсте России 11 декабря 2020 г. N 61391       ФЕДЕРАЛЬНАЯ СЛУЖБА ПО ЭКОЛОГИЧЕСКОМУ,  ТЕХНОЛОГИЧЕСКОМУ И АТОМНОМУ НАДЗОРУ    ПРИКАЗ  от 20 октября 2020 г. N 420    ОБ УТВЕРЖДЕНИИ ФЕДЕРАЛЬНЫХ НОРМ И  ПРАВИЛ В ОБЛАСТИ ПРОМЫШЛЕННОЙ  БЕЗОПАСНОСТИ "ПРАВИЛА ПРОВЕДЕНИЯ  ЭКСПЕРТИЗЫ ПРОМЫШЛЕННОЙ  БЕЗОПАСНОСТИ"  (в ред. Приказа Ростехнадзора от 13.04.2022 N 120)    В соответствии со статьей 5 Федерального закона от 21 июля 1997 г. N 116-ФЗ "О  промышленной  безопасности  опасных  производственных  объектов"  (Собрание  законодательства Российской Федерации, 1997, N 30, ст. 3588; 2018, N 31, ст. 4860),  подпунктом 5.2.2.16(1) пункта 5 Положения о Федеральной службе по экологическому,  технологическому и атомному надзору, утвержденного постановлением Правительства  Российской Федерации от 30 июля 2004 г. N 401 "О Федеральной службе по

In [129]:
random.sample(pages_and_texts, k=3)

[{'page_number': 1,
  'page_char_count': 1362,
  'page_word_count': 223,
  'page_sentence_count_row': 16,
  'page_token_count': 340.5,
  'text': 'Зарегистрировано в Минюсте России 11 декабря 2020 г. N 61391       ФЕДЕРАЛЬНАЯ СЛУЖБА ПО ЭКОЛОГИЧЕСКОМУ,  ТЕХНОЛОГИЧЕСКОМУ И АТОМНОМУ НАДЗОРУ    ПРИКАЗ  от 20 октября 2020 г. N 420    ОБ УТВЕРЖДЕНИИ ФЕДЕРАЛЬНЫХ НОРМ И  ПРАВИЛ В ОБЛАСТИ ПРОМЫШЛЕННОЙ  БЕЗОПАСНОСТИ "ПРАВИЛА ПРОВЕДЕНИЯ  ЭКСПЕРТИЗЫ ПРОМЫШЛЕННОЙ  БЕЗОПАСНОСТИ"  (в ред. Приказа Ростехнадзора от 13.04.2022 N 120)    В соответствии со статьей 5 Федерального закона от 21 июля 1997 г. N 116-ФЗ "О  промышленной  безопасности  опасных  производственных  объектов"  (Собрание  законодательства Российской Федерации, 1997, N 30, ст. 3588; 2018, N 31, ст. 4860),  подпунктом 5.2.2.16(1) пункта 5 Положения о Федеральной службе по экологическому,  технологическому и атомному надзору, утвержденного постановлением Правительства  Российской Федерации от 30 июля 2004 г. N 401 "О Федеральной службе по

In [130]:
dataframe = pd.DataFrame(pages_and_texts)

In [131]:
dataframe

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count,text
0,1,1362,223,16,340.50,Зарегистрировано в Минюсте России 11 декабря 2...
1,2,2158,301,16,539.50,и атомному надзору от 20 октября 2020 г. N 42...
2,3,2135,310,18,533.75,<1> Пункт 2 статьи 7 Федерального закона от 21...
3,4,2158,305,9,539.50,2) иметь стаж работы не менее 10 лет по специа...
4,5,2209,310,18,552.25,"основании организации, в трудовых отношениях с..."
5,6,2480,347,20,620.00,проведению экспертизы промышленной безопасност...
6,7,2577,336,15,644.25,<13> Пункт 3 статьи 2 Федерального закона от 2...
7,8,2343,289,7,585.75,"расследования аварий и инцидентов, связанных с..."
8,9,2735,366,8,683.75,а) определение соответствия строительных конст...
9,10,2309,307,13,577.25,выполнению этих работ и учитывать результаты р...


In [133]:
dataframe.describe().round()

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count
count,12.0,12.0,12.0,12.0,12.0
mean,6.0,2232.0,307.0,13.0,558.0
std,4.0,385.0,44.0,4.0,96.0
min,1.0,1362.0,223.0,7.0,340.0
25%,4.0,2152.0,298.0,10.0,538.0
50%,6.0,2259.0,308.0,14.0,565.0
75%,9.0,2504.0,339.0,16.0,626.0
max,12.0,2735.0,366.0,20.0,684.0


### Text Processing (Splitting pages into sentences )

In [166]:
import spacy
from spacy.lang.ru import Russian

nlp = Russian()
nlp.add_pipe('sentencizer')

doc = nlp("Это предложение. Это другое предложение. дааа")
assert len(list(doc.sents)) == 3
list(doc.sents)

[Это предложение., Это другое предложение., дааа]

In [161]:
from spacy.lang.ru import Russian

nlp = Russian()
nlp.add_pipe("sentencizer")
doc = nlp("Это предложение. Это другое предложение.")
assert len(list(doc.sents)) == 2

In [162]:
list(doc.sents)

[Это предложение., Это другое предложение.]

In [169]:
for item in tqdm(pages_and_texts):
    item["sentence"] = list(nlp(item["text"]).sents)
    
    item["sentence"] = [str(sentence) for sentence in item["sentence"]]
    
    item["page_sentence_count_spacy"] = len(item['sentence'])

100%|██████████| 12/12 [00:00<00:00, 59.48it/s]


In [180]:
df = pd.DataFrame(pages_and_texts)

df.describe().round()

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count,page_sentence_count_spacy,num_chunks
count,12.0,12.0,12.0,12.0,12.0,12.0,12.0
mean,6.0,2232.0,307.0,13.0,558.0,11.0,2.0
std,4.0,385.0,44.0,4.0,96.0,4.0,1.0
min,1.0,1362.0,223.0,7.0,340.0,5.0,1.0
25%,4.0,2152.0,298.0,10.0,538.0,9.0,2.0
50%,6.0,2259.0,308.0,14.0,565.0,10.0,2.0
75%,9.0,2504.0,339.0,16.0,626.0,13.0,2.0
max,12.0,2735.0,366.0,20.0,684.0,17.0,3.0


In [170]:
random.sample(pages_and_texts, k=1)

[{'page_number': 8,
  'page_char_count': 2343,
  'page_word_count': 289,
  'page_sentence_count_row': 7,
  'page_token_count': 585.75,
  'text': 'расследования аварий и инцидентов, связанных с эксплуатацией технических устройств,  заключения экспертизы ранее проводимых экспертиз) и режимам эксплуатации  технических устройств (при наличии);  б) расчетные и аналитические процедуры оценки и прогнозирования технического  состояния технических устройств (в случаях, при которых проводится техническое  диагностирование технических устройств).  25. Техническое диагностирование технических устройств включает следующие  мероприятия:  а) визуальный и измерительный контроль;  б) оперативное (функциональное) диагностирование для получения информации о  состоянии, фактических параметрах работы, фактического нагружения технического  устройства в реальных условиях эксплуатации;  в) определение действующих повреждающих факторов, механизмов повреждения и  восприимчивости материала технического устройств

### Chunking Sentences 

In [175]:
num_sentence_chunk_size = 7

def split_list(input_list: list[str], slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

In [174]:
test_list=list(range(10))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]

In [178]:
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentence"], slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

100%|██████████| 12/12 [00:00<00:00, 12090.23it/s]


In [179]:
df = pd.DataFrame(pages_and_texts)
df.describe().round()

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count,page_sentence_count_spacy,num_chunks
count,12.0,12.0,12.0,12.0,12.0,12.0,12.0
mean,6.0,2232.0,307.0,13.0,558.0,11.0,2.0
std,4.0,385.0,44.0,4.0,96.0,4.0,1.0
min,1.0,1362.0,223.0,7.0,340.0,5.0,1.0
25%,4.0,2152.0,298.0,10.0,538.0,9.0,2.0
50%,6.0,2259.0,308.0,14.0,565.0,10.0,2.0
75%,9.0,2504.0,339.0,16.0,626.0,13.0,2.0
max,12.0,2735.0,366.0,20.0,684.0,17.0,3.0


In [198]:
import re

pages_chunks = []

for item in tqdm(pages_and_texts):
    for sentence_chunk in item['sentence_chunks']:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        joined_sentence_chunk = "".join(sentence_chunk).replace(" ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r' . \1', joined_sentence_chunk)
        
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        
        pages_chunks.append(chunk_dict)

len(pages_chunks)
        

100%|██████████| 12/12 [00:00<00:00, 408.05it/s]


24

In [200]:
random.sample(pages_chunks, k=1)

[{'page_number': 7,
  'sentence_chunk': '<13> Пункт 3 статьи 2 Федерального закона от 21 июля 1997 г. N 116-ФЗ "О  промышленной безопасности опасных производственных объектов".   21.Экспертная организация приступает к проведению экспертизы после:  предоставления заказчиком необходимых для проведения экспертизы документов;  предоставления образцов технических устройств либо обеспечения доступа экспертов к  техническим устройствам, зданиям и сооружениям, применяемым на опасном  производственном объекте. 22.Заказчик обязан предоставить безопасный доступ экспертам, участвующим в  проведении экспертизы, к техническим устройствам, применяемым на опасном  производственном объекте, к зданиям и сооружениям опасных производственных  объектов, в отношении которых проводится экспертиза. Эксперты, участвующие в проведении экспертизы, обязаны соблюдать положения  нормативных  правовых  актов,  устанавливающих  требования  промышленной  безопасности, а также правила ведения работ на опасном производс

In [201]:
df = pd.DataFrame(pages_chunks)

df.describe().round()

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,24.0,24.0,24.0,24.0
mean,7.0,1189.0,159.0,297.0
std,3.0,639.0,84.0,160.0
min,1.0,68.0,8.0,17.0
25%,4.0,868.0,112.0,217.0
50%,6.0,1249.0,162.0,312.0
75%,9.0,1432.0,198.0,358.0
max,12.0,2728.0,359.0,682.0


In [207]:
# Show random chunks with few tokens

min_token_length = 15
for row in df[df["chunk_token_count"] <= min_token_length].sample(3).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

ValueError: a must be greater than 0 unless no samples are taken